# Web Scraping - Bomtempo

In [1]:
import time

import undetected_chromedriver as uc

from selenium.webdriver.remote.webdriver import By
#import selenium.webdriver.support.expected_conditions as EC  # noqa
#from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException


In [2]:
def set_viewport_size(driver, width, height):
    window_size = driver.execute_script("""
        return [window.outerWidth - window.innerWidth + arguments[0],
          window.outerHeight - window.innerHeight + arguments[1]];
        """, width, height)
    driver.set_window_size(*window_size)

In [3]:
driver = uc.Chrome()
set_viewport_size(driver, 400, 900)
driver.get("https://loja.bontemposupermercados.com/Bontempo/mercearia-seca-59/")

In [4]:
menuCategoryBttn = driver.find_element(By.XPATH, "/html/body/section[2]/div/div[1]/div[2]")
menuCategoryBttn.click()

In [5]:
categoriesLinks = []

categoriesDivList = driver.find_elements(By.XPATH, "/html/body/section[2]/div/div[2]/*") 

for div in categoriesDivList:
    link = div.find_element(By.TAG_NAME, "a").get_attribute("href")
    categoriesLinks.append(link)
    
categoriesLinks

['https://loja.bontemposupermercados.com/Bontempo/mercearia-seca-59/',
 'https://loja.bontemposupermercados.com/Bontempo/bebidas-88/',
 'https://loja.bontemposupermercados.com/Bontempo/pereciveis-85/',
 'https://loja.bontemposupermercados.com/Bontempo/bonjour-158/',
 'https://loja.bontemposupermercados.com/Bontempo/hortifruti-116/',
 'https://loja.bontemposupermercados.com/Bontempo/mercearia-salgada-50/',
 'https://loja.bontemposupermercados.com/Bontempo/mercearia-doce-47/',
 'https://loja.bontemposupermercados.com/Bontempo/limpeza-3/',
 'https://loja.bontemposupermercados.com/Bontempo/perfumaria-10/',
 'https://loja.bontemposupermercados.com/Bontempo/cosmeticos-1/',
 'https://loja.bontemposupermercados.com/Bontempo/suplementos-alimentares-517/',
 'https://loja.bontemposupermercados.com/Bontempo/bazar-32/',
 'https://loja.bontemposupermercados.com/Bontempo/eletro-123/']

In [11]:
def getProximoButton():
	try:
		bttn = driver.find_element(By.CSS_SELECTOR, "nav.pagination a.btn.btn-outline-secondary.btn-sm")
		return bttn if bttn.text == 'Próxima ' else None
	except NoSuchElementException:
		return None

getProximoButton()

<undetected_chromedriver.webelement.WebElement (session="db2f9b030dd44880afa453758c90b5a6", element="DABB7E41D52E00CD91483FD31553CC05_element_601")>

In [14]:
productList = []

baseLink = "https://loja.bontemposupermercados.com/Bontempo/mercearia-seca-59/"
proximoButton = True

# Para buscar também as diferentes categorias:

# for link in categoriesLinks:
#   driver.get(link)
#   time.sleep(2)

while (proximoButton != None):

    # driver.get(link)

    productGrid = driver.find_elements(By.XPATH, "/html/body/section[3]/div/div/div[1]/div[2]/*")
        
    for product in productGrid:

        try:
            mediaDiv = product.find_element(By.CSS_SELECTOR, "div>div>div.media")
        except NoSuchElementException: 
            break

        captionDiv = product.find_element(By.CSS_SELECTOR, "div>div>div.caption")

        linkDiv = mediaDiv.find_element(By.TAG_NAME, "a")

        name = captionDiv.find_element(By.CSS_SELECTOR, "a.link-nome-produto > span.nome").text
        price = captionDiv.find_element(By.CSS_SELECTOR, "div.detail > div.container-preco > input").get_attribute("value")
        link = linkDiv.get_attribute("href")
        image = linkDiv.find_element(By.TAG_NAME, "img").get_attribute("data-src")

        productList.append({"name": name, "price": price, "link": link, "image": image})
    
    proximoButton = getProximoButton()
    if (proximoButton != None):
        proximoButton.click()
        time.sleep(2)

productList

[{'name': 'Tapioca Amafil 500g Media',
  'price': '7,69',
  'link': 'https://loja.bontemposupermercados.com/Bontempo/produto/tapioca-amafil-500g-media-2205/',
  'image': 'https://storage.googleapis.com/phygital_files/bontempo/uploads/produto/tapioca_amafil_500g_media_d833d537-c46d-438c-b033-69fc04122f4b.thumbnail.jpg'},
 {'name': 'Tapioca Amafil 500g Grossa',
  'price': '7,69',
  'link': 'https://loja.bontemposupermercados.com/Bontempo/produto/tapioca-amafil-500g-grossa-4341/',
  'image': 'https://storage.googleapis.com/phygital_files/bontempo/uploads/produto/tapioca_amafil_500g_grossa_8ac104d0-6940-4159-b70d-f3d1a9c75208.thumbnail.jpg'},
 {'name': 'Sal Rn 1kg',
  'price': '0,99',
  'link': 'https://loja.bontemposupermercados.com/Bontempo/produto/sal-rn-1kg-204/',
  'image': 'https://storage.googleapis.com/phygital_files/bontempo/uploads/produto/sal_rn_1kg_4b55046f-d64a-4575-a4f8-5cef867c8492.thumbnail.jpg'},
 {'name': 'Sal Master 1kg',
  'price': '1,29',
  'link': 'https://loja.bontem

In [20]:
if (len(productList) > 0):
    import csv
    from os.path import exists

    csvPath = "data/bomtempo.csv"

    existsCsv = exists(csvPath)

    fieldnames = productList[0].keys()

    if (not existsCsv):
        with open(csvPath, 'w', encoding='UTF8', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(productList)
    else:
        raise FileExistsError(f"Arquivo {csv} já existe!")
else:
    raise AssertionError("A lista de produtos extraídos está vazia")
